In [1]:
import tensorflow as tf

## 0.前言
> 照抄原文

我们来特别地讨论网络的最后一层的设计,它除了和所有的隐藏层一样,完成维度变换、特征提取的功能,还作为输出层使用,需要根据具体的任务场景来决定是否使用激活函数,以及使用什么类型的激活函数等。
我们将根据输出值的区间范围来分类讨论。

常见的几种输出类型包括:
* $o_i\in R^d$输出属于整个实数空间,或者某段普通的实数空间,比如函数值趋势的预测,年龄的预测问题等。
* $o_i\in [0,1]$输出值特别地落在[0, 1]的区间,如图片生成,图片像素值一般用[0, 1]区间的值表示;或者二分类问题的概率,如硬币正反面的概率预测问题。
* $o_i\in [0,1],\Sigma_i\ o_i=1$输出值落在[0,1]的区间,并且所有输出值之和为 1,常见的如多分类问题,如 MNIST 手写数字图片识别,图片属于10个类别的概率之和应为1。
* $o_i\in [-1,1]$输出值在[-1, 1]之间

## 1.普通实数空间

针对问题：年龄预测、股票走势等

输出层可以不加激活函数，基于最后一层的输出$o$和真实值$y$直接计算误差：

${\cal L}=g(o,y)$

## 2.[0,1]区间

针对的问题：图片的生成、二分类问题等。

输出层通常采用Sigmoid函数。

## 3.[0,1]区间，和为1

针对问题：多分类。

输出层通常采用Softmax函数。

下图是一个处理3分类任务的网络结构：

![](https://github.com/zfhxi/Learn_tensorflow/blob/master/ch06-%C9%F1%BE%AD%CD%F8/img/011.png?raw=true)

假设有如下的例子，输出层输出为[2.,1.,.1]，经softmax计算后得到输出为[.7,.2,.1]:

![](https://github.com/zfhxi/Learn_tensorflow/blob/master/ch06-%C9%F1%BE%AD%CD%F8/img/012.png?raw=true)

在TensorFlow中实现如下：

In [2]:
z=tf.constant([2.,1.,.1])
y=tf.nn.softmax(z)
print(y)

tf.Tensor([0.6590012 0.242433  0.0985659], shape=(3,), dtype=float32)


与Dense层类似，Softmax函数也可以作为网络层使用，通过类`layers.Softmax(axis=-1)`可以方便添加，axis参数指定需要计算的维度

在Softmax函数的数值计算过程中，易因输入值偏大发生数值溢出现象；在计算交叉熵时，也会出现数值溢出问题。

为了数值计算的稳定性，TensorFlow提供了一个统一的接口，将Softmax与交叉熵损失函数同时实现，同时处理数值不稳定的异常。
接口为：

`tf.keras.losses.categorical_crossentropy(y_true,y_pred,from_logits=False)`
* y_true代表one-hot编码后的真实标签
* y_pred表示网络的预测值
* from_logits设置为True时，y_pred必须为没有经过Softmax函数的变量z；设置为False时，y_pred必须已经经过Softmax处理。

为了数值计算的稳定性，一般`from_logits`为True，此时`tf.keras.losses.categorical_crossentropy(y_true,y_pred,from_logits=True)`在内部进行Softmax计算，不需要在模型中显式调用Softmax函数。例如：


In [3]:
z=tf.random.normal([2,10])
y=tf.constant([1,3])
y_onehot=tf.one_hot(y,depth=10) #one-hot编码

# 输出层未使用softmax函数，故from_logits设置为True
loss=tf.keras.losses.categorical_crossentropy(y_onehot,z,from_logits=True)
loss=tf.reduce_mean(loss)
print(loss)

tf.Tensor(3.160231, shape=(), dtype=float32)


除了使用`tf.keras.losses.categorical_crossentropy(y_true,y_pred,from_logits=False)`这种函数式接口，也可使用类接口：

`tf.keras.losses.CategoricalCrossentropy(from_logits=True)`来同时计算Softmax和交叉熵损失函数。如：
``

In [4]:
criteon=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
loss=criteon(y_onehot,z)
print(loss)

tf.Tensor(3.160231, shape=(), dtype=float32)


## 4.[-1,1]
如果希望输出值的范围分布在(-1,1)区间，可以简单地使用tanh函数，实现如下：

In [5]:
x=tf.linspace(-6.,6.,10)
y=tf.tanh(x)
print(y)

tf.Tensor(
[-0.9999877  -0.99982315 -0.997458   -0.9640276  -0.58278286  0.5827831
  0.9640276   0.997458    0.99982315  0.9999877 ], shape=(10,), dtype=float32)


In [ ]:
import os
pid=os.getpid()
!kill -9 $pid